In [ ]:
# TODO SAVE PER EPOCH IN NON UNBALANCED SET
import tensorflow as tf
import numpy as np
import h5py
import math
from math import sqrt
import argparse
import os
import random

# Generates and returns the VGG graph (up to the predictions node)
def Gen_VGG(COCHLEAGRAM_LENGTH, numlabels, train_mean_coch, padding="SAME"):
    final_filter = 512
    full_length = final_filter * math.ceil(COCHLEAGRAM_LENGTH / 171 / 2 / 2 / 2 / 2 / 2) * math.ceil(
        171 / 2 / 2 / 2 / 2 / 2)
    print(full_length)
    nets = {}
    with tf.name_scope('input'):
        nets['input_to_net'] = tf.placeholder(tf.float32, [None, COCHLEAGRAM_LENGTH], name='input_to_net')
        mean_tensor = tf.constant(train_mean_coch, dtype=tf.float32)
        nets['subtract_mean'] = tf.subtract(nets['input_to_net'], mean_tensor, name='Subtract_Mean')
        nets['reshapedCoch'] = tf.reshape(nets['subtract_mean'], [-1, 171, int(COCHLEAGRAM_LENGTH / 171), 1],
                                          name='reshape_input')

    with tf.name_scope("accuracy"):
        nets['accuracy'] = tf.placeholder(tf.float32, (), name='acc')
        nets['accsum'] = tf.summary.scalar("accuracy", nets['accuracy'])

    # sess.run(tf.global_variables_initializer())
    # thing = sess.run(reshapedCoch,feed_dict={input_to_net:train_coch[0:2],actual_labels:trainlabels[0:2]})

    with tf.variable_scope('conv1') as scope:
        # Conv_1_1 and save the graphs
        tf.get_variable_scope().reuse_variables()
        nets['conv1_1_Weights'] = weight_variable([3, 3, 1, 64])

        nets['layer1'] = variable_summaries(nets['conv1_1_Weights'])

        nets['grid'] = put_kernels_on_grid(nets['conv1_1_Weights'], 8, 8)
        nets['conv1_weight_image'] = tf.summary.image('conv1/kernels', nets['grid'], max_outputs=3)
        nets['conv1_1'] = conv2d(nets['reshapedCoch'], Weights=nets['conv1_1_Weights'], bias=bias_variable([64]),
                                 strides=1, name='conv1_1')

        # conv_1_2
        nets['conv1_2'] = conv2d(nets['conv1_1'], Weights=weight_variable([3, 3, 64, 64]), bias=bias_variable([64]),
                                 strides=1, name='conv1_2')

    with tf.name_scope("maxpool1"):
        nets['maxpool1'] = maxpool2x2(nets['conv1_2'], k=2, name='maxpool1', padding=padding)

    with tf.name_scope('conv2'):
        nets['conv2_1'] = conv2d(nets['maxpool1'], Weights=weight_variable([3, 3, 64, 128]), bias=bias_variable([128]),
                                 strides=1, name='conv2_1')
        nets['conv2_2'] = conv2d(nets['conv2_1'], Weights=weight_variable([3, 3, 128, 128]), bias=bias_variable([128]),
                                 strides=1, name='conv2_2')
    with tf.name_scope("maxpool2"):
        nets['maxpool2'] = maxpool2x2(nets['conv2_2'], k=2, name='maxpool2', padding=padding)

    with tf.name_scope('conv3'):
        nets['conv3_1'] = conv2d(nets['maxpool2'], Weights=weight_variable([3, 3, 128, 256]), bias=bias_variable([256]),
                                 strides=1, name='conv3_1')
        nets['conv3_2'] = conv2d(nets['conv3_1'], Weights=weight_variable([3, 3, 256, 256]), bias=bias_variable([256]),
                                 strides=1, name='conv3_2')
        nets['conv3_3'] = conv2d(nets['conv3_2'], Weights=weight_variable([3, 3, 256, 256]), bias=bias_variable([256]),
                                 strides=1, name='conv3_3')
        nets['conv3_4'] = conv2d(nets['conv3_3'], Weights=weight_variable([3, 3, 256, 256]), bias=bias_variable([256]),
                                 strides=1, name='conv3_4')
    with tf.name_scope("maxpool3"):
        nets['maxpool3'] = maxpool2x2(nets['conv3_4'], k=2, name='maxpool3', padding=padding)

    with tf.name_scope('conv4'):
        nets['conv4_1'] = conv2d(nets['maxpool3'], Weights=weight_variable([3, 3, 256, 512]), bias=bias_variable([512]),
                                 strides=1, name='conv4_1')
        nets['conv4_2'] = conv2d(nets['conv4_1'], Weights=weight_variable([3, 3, 512, 512]), bias=bias_variable([512]),
                                 strides=1, name='conv4_2')
        nets['conv4_3'] = conv2d(nets['conv4_2'], Weights=weight_variable([3, 3, 512, 512]), bias=bias_variable([512]),
                                 strides=1, name='conv4_3')
        nets['conv4_4'] = conv2d(nets['conv4_3'], Weights=weight_variable([3, 3, 512, 512]), bias=bias_variable([512]),
                                 strides=1, name='conv4_4')

    with tf.name_scope("maxpool4"):
        nets['maxpool4'] = maxpool2x2(nets['conv4_4'], k=2, name='maxpool4', padding=padding)

    with tf.name_scope('conv5'):
        nets['conv5_1'] = conv2d(nets['maxpool4'], Weights=weight_variable([3, 3, 512, 512]), bias=bias_variable([512]),
                                 strides=1, name='conv5_1')
        nets['conv5_2'] = conv2d(nets['conv5_1'], Weights=weight_variable([3, 3, 512, 512]), bias=bias_variable([512]),
                                 strides=1, name='conv5_2')
        nets['conv5_3'] = conv2d(nets['conv5_2'], Weights=weight_variable([3, 3, 512, 512]), bias=bias_variable([512]),
                                 strides=1, name='conv5_3')
        nets['conv5_4'] = conv2d(nets['conv5_3'], Weights=weight_variable([3, 3, 512, 512]), bias=bias_variable([512]),
                                 strides=1, name='conv5_4')

    with tf.name_scope("maxpool5"):
        nets['maxpool5'] = maxpool2x2(nets['conv5_4'], k=2, name='maxpool5', padding=padding)

    with tf.name_scope("flatten"):
        nets['flattened'] = tf.reshape(nets['maxpool5'], [-1, full_length])

    with tf.name_scope("keep_prob"):
        nets['keep_prob'] = tf.placeholder(tf.float32)

    with tf.name_scope('fc_1'):
        W_fc1 = weight_variable([full_length, 4096])  # 4,959,232
        b_fc1 = bias_variable([4096])
        nets['fc_1'] = tf.nn.relu(tf.matmul(nets['flattened'], W_fc1) + b_fc1)
        nets['h_fc1_drop'] = tf.nn.dropout(nets['fc_1'], nets['keep_prob'])

    with tf.name_scope('fc_2'):
        W_fc2 = weight_variable([4096, 4096])  # 4,959,232
        b_fc2 = bias_variable([4096])
        nets['fc_2'] = tf.nn.relu(tf.matmul(nets['h_fc1_drop'], W_fc2) + b_fc2)
        nets['h_fc2_drop'] = tf.nn.dropout(nets['fc_2'], nets['keep_prob'])

    with tf.name_scope('fc_3'):
        W_fc3 = weight_variable([4096, numlabels], name='W_fc3')  # 4,959,232
        b_fc3 = bias_variable([numlabels], name='b_fc3')
        nets['predicted_labels'] = tf.nn.relu(tf.matmul(nets['h_fc2_drop'], W_fc3) + b_fc3)

    return nets


# Generates and returns our own graph (up to the predictions node)
def Gen_audiosetNet(COCHLEAGRAM_LENGTH, numlabels, train_mean_coch, Conv1_filtersize=9, padding="SAME",
                    poolmethod="MAXPOOL"):
    variable_list = []

    conv1_strides = 3
    conv2_strides = 2
    conv3_strides = 1
    conv4_strides = 1
    conv5_strides = 1
    final_filter = 512
    full_length = final_filter * math.ceil(
        COCHLEAGRAM_LENGTH / 171 / conv1_strides / conv2_strides / conv3_strides / conv4_strides / conv5_strides / 4 / 2) * math.ceil(
        171 / conv1_strides / conv2_strides / conv3_strides / conv4_strides / conv5_strides / 4 / 2)

    nets = {}
    with tf.name_scope('input'):
        nets['input_to_net'] = tf.placeholder(tf.float32, [None, COCHLEAGRAM_LENGTH], name='input_to_net')
        mean_tensor = tf.constant(train_mean_coch, dtype=tf.float32)
        nets['subtract_mean'] = tf.subtract(nets['input_to_net'], mean_tensor, name='Subtract_Mean')
        nets['reshapedCoch'] = tf.reshape(nets['subtract_mean'], [-1, 171, int(COCHLEAGRAM_LENGTH / 171), 1],
                                          name='reshape_input')

        # print("input ", nets['reshapedCoch'].shape)

    with tf.name_scope("images"):
        nets['coch_images'] = tf.summary.image('image', nets['reshapedCoch'][0:5, :, :, :])

    with tf.name_scope("accuracy"):
        nets['accuracy'] = tf.placeholder(tf.float32, (), name='acc')
        nets['accsum'] = tf.summary.scalar("accuracy", nets['accuracy'])

    with tf.variable_scope('conv1') as scope:
        tf.get_variable_scope().reuse_variables()
        nets['conv1_Weights'] = weight_variable([Conv1_filtersize, Conv1_filtersize, 1, 96], name='conv1_Weights')
        nets['conv1_bias'] = bias_variable([96], name='bias1_Weights')
        nets['layer1'] = variable_summaries(nets['conv1_Weights'])
        nets['grid'] = put_kernels_on_grid(nets['conv1_Weights'], 16, 6)
        nets['conv1_weight_image'] = tf.summary.image('conv1/kernels', nets['grid'], max_outputs=3)
        nets['conv1'] = tf.nn.local_response_normalization(
            conv2d(nets['reshapedCoch'], Weights=nets['conv1_Weights'], bias=nets['conv1_bias'], strides=conv1_strides,
                   name='conv1', padding=padding), depth_radius=2)

    with tf.name_scope("conv1_summaries"):
        sum_list = []
        for im in range(2):
            for channel in range(2):
                sum_list.append(tf.summary.image('conv1/featuremaps_im_{0}_ch{0}'.format(im, channel),
                                                 nets['conv1'][im:im + 1, :, :, channel:channel + 1], max_outputs=3))
                sum_list.append(tf.summary.image('conv1/weight_im{0}_ch{0}'.format(im, channel),
                                                 tf.transpose(nets['conv1_Weights'], (2, 0, 1, 3))[0:1, :, :,
                                                 channel:channel + 1], max_outputs=3))
                sum_list.append(tf.summary.histogram('conv1/featuremaps_hist_im_{0}_ch{0}'.format(im, channel),
                                                     nets['conv1'][im:im + 1, :, :, channel:channel + 1]))
        nets['conv1_sums'] = tf.summary.merge(sum_list)

        # print("conv1", nets['conv1'].shape)

    if poolmethod == "MAXPOOL":
        with tf.name_scope("maxpool1"):
            nets['pool1'] = maxpool2x2(nets['conv1'], k=2, name='maxpool1', padding="SAME")
    elif poolmethod == "HPOOL":
        with tf.name_scope("h_pool1"):
            nets['pool1'] = add_hanning_pooling(nets, 'conv1', downsample=2, length_of_window=8, layer_name='pool1',
                                                make_plots=False)

            # print("pool1, ", nets['pool1'].shape)

    with tf.name_scope('conv2'):
        nets['conv2_Weights'] = weight_variable([5, 5, 96, 256], name='conv2_Weights')
        nets['conv2'] = tf.nn.local_response_normalization(
            conv2d(nets['pool1'], Weights=nets['conv2_Weights'], bias=bias_variable([256], name='conv2_bias'),
                   strides=conv2_strides, name='conv2', padding=padding), depth_radius=2)

        # print("conv2 ", nets['conv2'].shape)

    if poolmethod == "MAXPOOL":
        with tf.name_scope("maxpool2"):
            nets['pool2'] = maxpool2x2(nets['conv2'], k=2, name='maxpool2', padding="SAME")
    elif poolmethod == "HPOOL":
        with tf.name_scope("h_pool2"):
            nets['pool2'] = add_hanning_pooling(nets, 'conv2', downsample=2, length_of_window=8, layer_name='pool2',
                                                make_plots=False)

    # print("pool2, ", nets['pool2'].shape)

    with tf.name_scope('conv3'):
        nets['conv3'] = conv2d(nets['pool2'], Weights=weight_variable([3, 3, 256, 512], name='conv3_Weights'),
                               bias=bias_variable([512], name='conv3_bias'), strides=conv3_strides, name='conv3',
                               padding=padding)
        # print("conv3 ", nets['conv3'].shape)

    with tf.name_scope('conv4'):
        nets['conv4'] = conv2d(nets['conv3'], Weights=weight_variable([3, 3, 512, 1024], name='conv4_Weights'),
                               bias=bias_variable([1024], name='conv4_bias'), strides=conv4_strides, name='conv4',
                               padding=padding)
        # print("conv4", nets['conv4'].shape)

    with tf.name_scope('conv5'):
        nets['conv5'] = conv2d(nets['conv4'], Weights=weight_variable([3, 3, 1024, final_filter], name='conv5_Weights'),
                               bias=bias_variable([final_filter], name='conv5_bias'), strides=conv5_strides,
                               name='conv5', padding=padding)

        # print("conv5",nets['conv5'].shape )

    if poolmethod == "MAXPOOL":
        with tf.name_scope("maxpool3"):
            nets['pool3'] = maxpool2x2(nets['conv5'], k=2, name='maxpool3', padding="SAME")
    elif poolmethod == "HPOOL":
        with tf.name_scope("h_pool3"):
            nets['pool3'] = add_hanning_pooling(nets, 'conv5', downsample=2, length_of_window=8, layer_name='pool3',
                                                make_plots=False)

    with tf.name_scope("flatten"):
        nets['flattened'] = tf.reshape(nets['pool3'], [-1, full_length])

    with tf.name_scope('fc_1'):
        W_fc1 = weight_variable([full_length, 1024])  # 4,959,232
        b_fc1 = bias_variable([1024])
        nets['fc_1'] = tf.nn.relu(tf.matmul(nets['flattened'], W_fc1) + b_fc1)

        nets['keep_prob'] = tf.placeholder(tf.float32)
        nets['h_fc1_drop'] = tf.nn.dropout(nets['fc_1'], nets['keep_prob'])

    with tf.name_scope('fc_2'):
        W_fc2 = weight_variable([1024, numlabels])
        b_fc2 = bias_variable([numlabels])
        nets['predicted_labels'] = tf.matmul(nets['h_fc1_drop'], W_fc2) + b_fc2

    return nets


def Cross_Entropy_Train_on_Labels(nets, numlabels, Learning_Rate, multiple_labels=True):
    # The evaluation/training of the graph
    with tf.variable_scope("Cross_Entropy"):
        nets['actual_labels'] = tf.placeholder(tf.float32, [None, numlabels], name='actual_labels')

        if multiple_labels:
            nets['cross_entroy'] = tf.reduce_mean(
                tf.nn.sigmoid_cross_entropy_with_logits(labels=nets['actual_labels'], logits=nets['predicted_labels']))
        else:
            nets['cross_entroy'] = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(labels=nets['actual_labels'], logits=nets['predicted_labels']))
            # nets['cross_entroy'] = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels =nets['actual_labels'],logits = nets['predicted_labels'] ))

    with tf.variable_scope("TrainStep"):
        '''
        beta1=0.9,
        beta2=0.999,
        epsilon=1e-08,'''
        beta1 = 0.9
        beta2 = 0.999
        epsilon = 1e-08
        nets['global_step'] = tf.Variable(0, name='global_step', trainable=False)
        nets['train_step'] = tf.train.AdamOptimizer(Learning_Rate).minimize(nets['cross_entroy'],
                                                                            global_step=nets['global_step'])
        # nets['train_step'] = tf.train.AdamOptimizer(learning_rate=Learning_Rate,beta1=beta1,beta2=beta2,epsilon=epsilon).minimize(nets['cross_entroy'])

        # num = np.sum(nets['actual_labelsi'])
        # nets['accuracy'] = tf.metrics.mean(tf.nn.in_top_k(nets['predicted_labels'],nets['actual_labelsi'],2))

        # calculating accuracy
    with tf.name_scope("Summaries"):
        nets['label_indeces'] = tf.placeholder(tf.int32, [None, 15], name='label_indeces')

        _, nets['indeces'] = tf.nn.top_k(nets['predicted_labels'], 15)
        prediction_sums = tf.summary.histogram('predicted_labels_histogram', nets['indeces'])
        nets['numCorrect'] = tf.shape(tf.sets.set_intersection(nets['indeces'], nets['label_indeces'], False).values)[0]

        _, nets['Top_1_index'] = tf.nn.top_k(nets['predicted_labels'], 1)
        nets['numCorrect_top_pred'] = \
        tf.shape(tf.sets.set_intersection(nets['Top_1_index'], nets['label_indeces'], False).values)[0]

    with tf.name_scope("Cross_Entropy_Loss"):
        nets['xe_ave'] = tf.placeholder(tf.float32, (), name='xe_ave')
        nets['xesum'] = tf.summary.scalar("Test_cross_entropy_loss_sample", nets['xe_ave'])
        nets['cross_entropy_summary'] = tf.summary.scalar("Train_cross_entropy_loss", nets['cross_entroy'])

        nets['xe_avet'] = tf.placeholder(tf.float32, (), name='xe_avet')
        nets['xesumt'] = tf.summary.scalar("Train_cross_entropy_loss_sample", nets['xe_avet'])

    with tf.name_scope("Count_Tracker"):
        nets['current_epoch'] = tf.Variable(0, name='current_epoch', trainable=False, dtype=tf.int32)
        nets['increment_current_epoch'] = tf.assign(nets['current_epoch'], nets['current_epoch'] + 1)

        nets['current_step'] = tf.Variable(0, name='current_step', trainable=False, dtype=tf.int32)
        nets['increment_current_step'] = tf.assign(nets['current_step'], nets['current_step'] + 1)
        nets['reset_current_step'] = tf.assign(nets['current_step'], 0)


# thanks to https://gist.githubusercontent.com/kukuruza/03731dc494603ceab0c5/raw/3d708320145df0a962cfadb95b3f716b623994e0/gist_cifar10_train.py
def put_kernels_on_grid(kernel, grid_Y, grid_X, pad=1):
    '''Visualize conv. features as an image (mostly for the 1st layer).
    Place kernel into a grid, with some paddings between adjacent filters.

    Args:
      kernel:            tensor of shape [Y, X, NumChannels, NumKernels]
      (grid_Y, grid_X):  shape of the grid. Require: NumKernels == grid_Y * grid_X
                           User is responsible of how to break into two multiples.
      pad:               number of black pixels around each filter (between them)

    Return:
      Tensor of shape [(Y+2*pad)*grid_Y, (X+2*pad)*grid_X, NumChannels, 1].
    '''

    x_min = tf.reduce_min(kernel)
    x_max = tf.reduce_max(kernel)

    kernel1 = (kernel - x_min) / (x_max - x_min)

    # pad X and Y
    x1 = tf.pad(kernel1, tf.constant([[pad, pad], [pad, pad], [0, 0], [0, 0]]), mode='CONSTANT')

    # X and Y dimensions, w.r.t. padding
    Y = kernel1.get_shape()[0] + 2 * pad
    X = kernel1.get_shape()[1] + 2 * pad

    channels = kernel1.get_shape()[2]

    # put NumKernels to the 1st dimension
    x2 = tf.transpose(x1, (3, 0, 1, 2))
    # organize grid on Y axis
    x3 = tf.reshape(x2, tf.stack([grid_X, Y * grid_Y, X, channels]))  # 3

    # switch X and Y axes
    x4 = tf.transpose(x3, (0, 2, 1, 3))
    # organize grid on X axis
    x5 = tf.reshape(x4, tf.stack([1, X * grid_X, Y * grid_Y, channels]))  # 3

    # back to normal order (not combining with the next step for clarity)
    x6 = tf.transpose(x5, (2, 1, 3, 0))

    # to tf.image_summary order [batch_size, height, width, channels],
    #   where in this case batch_size == 1
    x7 = tf.transpose(x6, (3, 0, 1, 2))

    # scale to [0, 255] and convert to uint8
    return tf.image.convert_image_dtype(x7, dtype=tf.uint8)


# a bunch of neat stats we can summarize
def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        meansum = tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        stdv = tf.summary.scalar('stddev', stddev)
        maxsum = tf.summary.scalar('max', tf.reduce_max(var))
        minsum = tf.summary.scalar('min', tf.reduce_min(var))
        hist = tf.summary.histogram('histogram', var)

        return tf.summary.merge([meansum, stdv, maxsum, minsum, hist])


def main(Learning_Rate, limit=None, music_only=False, print_extras=True, Net_or_VGG="Net", name="", unbalanced=False,
         folder="TB2", Conv1_filtersize=9, TASK="PREDICT_LABELS", OVERIDE_FOLDER=None, SAVE=True, padding="SAME",
         poolmethod="MAXPOOL"):
    # if we want to use the unbalanced set for training/eval, read that file in
    if (unbalanced):
        trainfile = "/om/user/ribeirop/audiosetDL/unbalanced_stripped/unbalanced_train_segments_Coch.hdf5"
        train_averagefile = "/om/user/ribeirop/audiosetDL/unbalanced_stripped/unbalanced_train_segments_average.npy"
    else:
        trainfile = "/om/user/ribeirop/audiosetDL/balanced_stripped/balanced_train_segments_Coch.hdf5"
        train_averagefile = "/om/user/ribeirop/audiosetDL/balanced_stripped/balanced_train_segments_average.npy"

    testfile = "/om/user/ribeirop/audiosetDL/eval_stripped/eval_segments_Coch.hdf5"
    test_averagefile = "/om/user/ribeirop/audiosetDL/eval_stripped/eval_segments_average.npy"
    trainset = h5py.File(trainfile, "r")
    testset = h5py.File(testfile, "r")

    train_coch = trainset["/coch"]  # The cochleagrams are in here
    test_coch = testset["/coch"]

    #    unbalanced_label_counts = np.load("/home/ribeirop/OMFOLDER/audiosetDL/MODIFIED_LABELS/unbalanced_label_counts.npy")


    if (TASK == "PREDICT_LABELS"):
        multiple_labels = True
        if (music_only):  # If we want to use a label set without music
            if (unbalanced):
                trainlabels_file = "/home/ribeirop/OMFOLDER/audiosetDL/MODIFIED_LABELS/unbalanced_train_segments_nomusic_labels_only.hdf5"
            else:
                trainlabels_file = "/home/ribeirop/OMFOLDER/audiosetDL/MODIFIED_LABELS/balanced_train_segments_nomusic_labels_only.hdf5"

            testlabels_file = "/home/ribeirop/OMFOLDER/audiosetDL/MODIFIED_LABELS/eval_segments_nomusic_labels_only.hdf5"

            trainlabelsSet = h5py.File(trainlabels_file, "r")
            testlabelsSet = h5py.File(testlabels_file, "r")

            trainlabels = trainlabelsSet["/labels"]
            testlabels = testlabelsSet["/labels"]

            numlabels = 525
        else:
            trainlabels = trainset["/labels"]
            testlabels = testset["/labels"]

            numlabels = 527
    elif (TASK == "PREDICT_COUNTS"):
        multiple_labels = False
        if (unbalanced):
            trainlabels = np.load("/home/ribeirop/OMFOLDER/audiosetDL/MODIFIED_LABELS/unbalanced_label_counts.npy")
        else:
            trainlabels = np.load("/home/ribeirop/OMFOLDER/audiosetDL/MODIFIED_LABELS/balanced_label_counts.npy")

        testlabels = np.load("/home/ribeirop/OMFOLDER/audiosetDL/MODIFIED_LABELS/eval_label_counts.npy")
        numlabels = 15

    COCHLEAGRAM_LENGTH = int(342000)  # full is 342000
    train_mean_coch = np.load(train_averagefile)[0:COCHLEAGRAM_LENGTH]  # mean cochleagram
    # test_mean_coch = np.load(test_averagefile)

    Num_Epochs = 2000

    if (not limit == None):
        trainsize = limit
        testsize = limit
    else:
        trainsize = trainset.attrs.get("size")
        testsize = testset.attrs.get("size")
        limit = "full"

    # for unbalanced set
    cutoff = int(trainsize * .90)

    b = 0
    batchsize = 100

    for epoch_step in range(6, 15):
        epoch_step_folder = "EPOCH_SAVED/{0}/model.ckpt-{1}".format(OVERIDE_FOLDER, epoch_step)

        print(epoch_step_folder)

        print("building graph")
        # The training of the graphs are defined here.
        with tf.Graph().as_default():

            if OVERIDE_FOLDER == None:
                # Get the base graph
                if (Net_or_VGG == "Net"):
                    TensorBoard_Folder_train = './{4}/{0}_{6}_{1}_n{2}_lr{3}_conv1FS{5}_TRAIN'.format(Net_or_VGG, name,
                                                                                                      limit,
                                                                                                      Learning_Rate,
                                                                                                      folder,
                                                                                                      Conv1_filtersize,
                                                                                                      TASK)
                    TensorBoard_Folder_test = './{4}/{0}_{6}_{1}_n{2}_lr{3}_conv1FS{5}_TEST'.format(Net_or_VGG, name,
                                                                                                    limit,
                                                                                                    Learning_Rate,
                                                                                                    folder,
                                                                                                    Conv1_filtersize,
                                                                                                    TASK)
                    Saver_Folder = '/{0}_{5}_{1}_n{2}_lr{3}_conv1FS{4}'.format(Net_or_VGG, name, limit, Learning_Rate,
                                                                               Conv1_filtersize, TASK)
                    nets = Gen_audiosetNet(COCHLEAGRAM_LENGTH, numlabels, train_mean_coch, Conv1_filtersize, padding,
                                           poolmethod)
                else:
                    TensorBoard_Folder_train = './{4}/{0}_{4}_{1}_n{2}_lr{3}_TRAIN'.format(Net_or_VGG, name, limit,
                                                                                           Learning_Rate, folder, TASK)
                    TensorBoard_Folder_test = './{4}/{0}_{4}_{1}_n{2}_lr{3}_TEST'.format(Net_or_VGG, name, limit,
                                                                                         Learning_Rate, folder, TASK)
                    Saver_Folder = '/{0}_{4}_{1}_n{2}_lr{3}'.format(Net_or_VGG, name, limit, Learning_Rate, TASK)
                    nets = Gen_VGG(COCHLEAGRAM_LENGTH, numlabels, train_mean_coch)
            else:
                TensorBoard_Folder_train = './{0}/{1}_TRAIN'.format(folder, OVERIDE_FOLDER)
                TensorBoard_Folder_test = './{0}/{1}_TEST'.format(folder, OVERIDE_FOLDER)
                Saver_Folder = '/{0}'.format(OVERIDE_FOLDER)
                if (Net_or_VGG == "Net"):
                    nets = Gen_audiosetNet(COCHLEAGRAM_LENGTH, numlabels, train_mean_coch, Conv1_filtersize, padding,
                                           poolmethod)
                else:
                    nets = Gen_VGG(COCHLEAGRAM_LENGTH, numlabels, train_mean_coch)

            print(TensorBoard_Folder_train)
            print(TensorBoard_Folder_test)
            # Get the loss functions
            Cross_Entropy_Train_on_Labels(nets, numlabels, Learning_Rate, multiple_labels)

            merged = tf.summary.merge_all()

            # CALC MISSING CROSS ENTROPY



            saver = tf.train.Saver()
            with tf.Session() as sess:
                saver.restore(sess, epoch_step_folder)
                offset = 0#random.randint(0,100*2000)
                train_writer = tf.summary.FileWriter(
                    TensorBoard_Folder_train)  # tensorboard writters for both train and test accuracy
                test_writer = tf.summary.FileWriter(TensorBoard_Folder_test)

                # EVALUATION
                print("eval")
                # pretty much the same thing but no train step.
                # print("eval")
                # TEST SET
                startindex = cutoff + offset # start at the first item in the last 10%
                endindex = startindex + batchsize
                xe_sum = 0
                print(startindex)
                numbatches = 0
                while (endindex <= cutoff + 200 * batchsize + offset):  # trainsize):
                    numbatches = numbatches + 1  # TODO: just calculate this
                    # print(startindex)
                    if(numbatches%10 == 0):
                        print(numbatches)
                    if TASK == "PREDICT_LABELS":
                        xe = sess.run(nets['cross_entroy'], feed_dict={
                            nets['input_to_net']: train_coch[startindex:endindex][:, 0:COCHLEAGRAM_LENGTH],
                            nets['actual_labels']: trainlabels[startindex:endindex], nets['keep_prob']: 1})
                    else:
                        xe = sess.run(nets['cross_entroy'], feed_dict={
                            nets['input_to_net']: train_coch[startindex:endindex][:, 0:COCHLEAGRAM_LENGTH],
                            nets['actual_labels']: trainlabels[startindex:endindex], nets['keep_prob']: 1})

                    xe_sum =  xe_sum+ xe
                    startindex = endindex
                    endindex = startindex + batchsize

                # print(nets['global_step'].eval(session=sess))
                print("xesum ",xe_sum )
                print("nb ", numbatches)
                print("thing ",xe_sum / numbatches )
                xesummary = sess.run(nets['xesum'], feed_dict={nets['xe_ave']: xe_sum / numbatches})
                test_writer.add_summary(xesummary, nets['global_step'].eval(session=sess))

                # TRAIN
                print("train")
                offset = 0#random.randint(0,cutoff - 200*100)
                startindex = 0  + offset
                endindex = startindex + batchsize
                xe_sum = 0
                numbatches = 0
                # loop through the first 90% of the set     
                while (endindex <= batchsize * 200 + offset):
                    # Get the indeces as ints for the labels
                    numbatches = numbatches + 1  # TODO: just calculate this
                    if(numbatches%10 == 0):
                        print(numbatches)
                    if TASK == "PREDICT_LABELS":
                        xe = sess.run(nets['cross_entroy'], feed_dict={
                            nets['input_to_net']: train_coch[startindex:endindex][:, 0:COCHLEAGRAM_LENGTH],
                            nets['actual_labels']: trainlabels[startindex:endindex], nets['keep_prob']: .5})
                    else:
                        xe = sess.run(nets['cross_entroy'], feed_dict={
                            nets['input_to_net']: train_coch[startindex:endindex][:, 0:COCHLEAGRAM_LENGTH],
                            nets['actual_labels']: trainlabels[startindex:endindex], nets['keep_prob']: .5})

                    xe_sum =  xe_sum+ xe
                    startindex = endindex
                    endindex = startindex + batchsize

                # print(nets['global_step'].eval(session=sess))
                print("xesum ",xe_sum )
                print("nb ", numbatches)
                print("thing ",xe_sum / numbatches )
                xesummary = sess.run(nets['xesumt'], feed_dict={nets['xe_avet']: xe_sum / numbatches})
                train_writer.add_summary(xesummary, nets['global_step'].eval(session=sess))

    print("done")


def weight_variable(shape, name=None):
    initial = tf.truncated_normal(shape, stddev=0.001)
    return tf.Variable(initial, name=name)


def bias_variable(shape, name=None):
    initial = tf.constant(0.0, shape=shape)
    return tf.Variable(initial, name=name)


# builds the components
def conv2d(inputtensor, Weights, bias, strides=1, name=None, padding="SAME"):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(inputtensor, Weights, strides=[1, strides, strides, 1], padding=padding, name=name)
    x = tf.nn.bias_add(x, bias)
    return tf.nn.relu(x)


def maxpool2x2(x, k=2, name=None, padding="SAME"):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding=padding, name=name)


def add_hanning_pooling(nets, top_node, downsample=2, length_of_window=8, layer_name=False, make_plots=False):
    """
    Add a layer using a hanning kernel for pooling

    Parameters
    ----------
    nets : dictionary
        a dictionary containing the graph
    downsample : int
        proportion downsampling
    top_node : string
        specify the node after which the spectemp filters will be added and used as input for the FFT.
    layer_name : False or string
        name for the layer. If false appends "_hpool" to the top_node name
    Returns
    -------
    nets : dictionary
        a dictionary containing the graph, containing a pooling layer
    """
    if not layer_name:
        layer_name = top_node + '_hpool'

    n_channels = nets[top_node].get_shape().as_list()[3]
    nets['hanning_tensor_' + top_node] = make_hanning_kernel_tensor(n_channels, downsample=downsample,
                                                                    length_of_window=length_of_window,
                                                                    make_plots=make_plots)
    return tf.nn.depthwise_conv2d(nets[top_node], nets['hanning_tensor_' + top_node], [1, downsample, downsample, 1],
                                  'SAME', name=layer_name)


def make_hanning_kernel_tensor(n_channels, downsample=2, length_of_window=8, make_plots=False):
    """
    Make a tensor containing the symmetric 2d hanning kernel to use for the pooling filters
    For downsample=2, using length_of_window=8 gives a reduction of -24.131545969216841 at 0.25 cycles
    For downsample=3, using length_of_window=12 gives a reduction of -28.607805482176282 at 1/6 cycles

    Parameters
    ----------
    n_channels : int
        number of channels to copy the kernel into
    downsample : int
        proportion downsampling
    length_of_window : int
        how large of a window to use
    make_plots: boolean
        make plots of the filters

    Returns
    -------
    hanning_tensor : tensorflow tensor
        tensorflow tensor containing the hanning kernel with size [1 length_of_window length_of_window n_channels]

    """
    hanning_kernel = make_hanning_kernel(downsample=downsample, length_of_window=length_of_window,
                                         make_plots=make_plots)
    hanning_kernel = np.expand_dims(np.dstack([hanning_kernel.astype(np.float32)] * n_channels), axis=3)
    hanning_tensor = tf.constant(hanning_kernel)
    return hanning_tensor


def make_hanning_kernel(downsample=2, length_of_window=8, make_plots=False):
    """
    Make the symmetric 2d hanning kernel to use for the pooling filters
    For downsample=2, using length_of_window=8 gives a reduction of -24.131545969216841 at 0.25 cycles
    For downsample=3, using length_of_window=12 gives a reduction of -28.607805482176282 at 1/6 cycles

    Parameters
    ----------
    downsample : int
        proportion downsampling
    length_of_window : int
        how large of a window to use
    make_plots: boolean
        make plots of the filters

    Returns
    -------
    two_dimensional_kernel : numpy array
        hanning kernel in 2d to use as a kernel for filtering

    """

    window = 0.5 * (1 - np.cos(2 * np.pi * (np.arange(length_of_window)) / (length_of_window - 1)))
    A = np.fft.fft(window, 2048) / (len(window) / 2.0)
    freq = np.linspace(-0.5, 0.5, len(A))
    response = 20 * np.log10(np.abs(np.fft.fftshift(A / abs(A).max())))

    nyquist = 1 / (2 * downsample)
    ny_idx = np.where(np.abs(freq - nyquist) == np.abs(freq - nyquist).min())[0][0]
    two_dimensional_kernel = np.sqrt(np.outer(window, window))

    if make_plots:
        print(['Frequency response at ' + 'nyquist' + ' is ' + 'response[ny_idx]'])
        plt.figure()
        plt.plot(window)
        plt.title(r"Hanning window")
        plt.ylabel("Amplitude")
        plt.xlabel("Sample")
        plt.figure()
        plt.plot(freq, response)
        plt.axis([-0.5, 0.5, -120, 0])
        plt.title(r"Frequency response of the Hanning window")
        plt.ylabel("Normalized magnitude [dB]")
        plt.xlabel("Normalized frequency [cycles per sample]")
        plt.figure()
        plt.matshow(two_dimensional_kernel)
        plt.colorbar()
        plt.title(r"Two dimensional kernel")

    return two_dimensional_kernel


if __name__ == '__main__':
    debug = True  # if true, then use the default csv file to read from.
    if debug:
        learning_rate = 1e-5
        music_only = False  # currently also removing speech
        limit = None
        Net_or_VGG = "Net"
        name = "BBB_TEST5"
        unbalanced = True
        folder = "TB3"
        Conv1_filtersize = 9
        TASK = "PREDICT_LABELS"
        # TASK = "PREDICT_COUNTS"
        OVERIDE_FOLDER = "Net_UNBALANCED91_5_nfull_lr1e-05"
        #OVERIDE_FOLDER =  "Net_UNBALANCED91_5_nfull_lr1e-05"
        SAVE = False
        padding = "SAME"
        # padding = "VALID"
        # poolmethod ="HPOOL"
        poolmethod = "MAXPOOL"
        main(learning_rate, limit, music_only, Net_or_VGG=Net_or_VGG, name=name, unbalanced=unbalanced, folder=folder,
             Conv1_filtersize=Conv1_filtersize, TASK=TASK, OVERIDE_FOLDER=OVERIDE_FOLDER, SAVE=SAVE, padding=padding,
             poolmethod=poolmethod)

    else:

        parser = argparse.ArgumentParser()
        parser.add_argument('-a', '--learning_rate', default=1e-6)
        parser.add_argument('-b', '--limit', default=None)
        parser.add_argument('-c', '--Model', default="Net")
        parser.add_argument('-n', '--name', default="noname")
        parser.add_argument('-f', '--conv1filtersize', default=9)
        parser.add_argument('-t', '--TASK', default="PREDICT_LABELS")
        parser.add_argument('-o', '--OVERIDE_FOLDER', default=None)

        args = vars(parser.parse_args())

        learning_rate = float(args["learning_rate"])

        if (not args["limit"] == None):
            limit = int(args["limit"])
        else:
            limit = None

        Net_or_VGG = args["Model"]
        name = args['name']
        Conv1_filtersize = int(args['conv1filtersize'])
        TASK = args['TASK']

        if (not args['OVERIDE_FOLDER'] == None):
            OVERIDE_FOLDER = args['OVERIDE_FOLDER']
        else:
            OVERIDE_FOLDER = None

        music_only = False
        unbalanced = True
        folder = "TB3"
        poolmethod = "MAXPOOL"
        padding = "SAME"
        SAVE = False

        main(learning_rate, limit, music_only, Net_or_VGG=Net_or_VGG, name=name, unbalanced=unbalanced, folder=folder,Conv1_filtersize=Conv1_filtersize, TASK=TASK, OVERIDE_FOLDER=OVERIDE_FOLDER, SAVE=SAVE, padding=padding,poolmethod=poolmethod)
